In [1]:
%matplotlib inline
from __future__ import print_function
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

In [2]:
def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
        conv = tf.nn.relu(conv)
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv

In [3]:
def SoftWeightLayer(inCh,outCh):   
    
    fc1 = tf.layers.dense(inCh,outCh,activation = None )    # doing a linear transformation(no activation applied)
    bn = tf.layers.batch_normalization(fc1)                                              
    
    return bn 

In [4]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        
        net = conv_block(x, h_dim, name='conv_1')
        net = conv_block(net, h_dim, name='conv_2')
        net = conv_block(net, h_dim, name='conv_3')
        net = conv_block(net, z_dim-1, name='conv_4') #need to change z_dim-1 to z_dim
        fx = tf.contrib.layers.flatten(net)      #need to reshape zdim to be the same dimensions as fx                                  
        wfx = SoftWeightLayer(fx,z_dim)  #then need to pass z_dim+1 and the reshaped z_dim in here
        
        return wfx,fx

In [5]:
def euclidean_distance(a, b):
    # a.shape = N x D
    # b.shape = M x D
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)

In [6]:
def compute_alpha(x, yw):
    
    # x: class*query X features
    # yw: class X weights + bias
    
    n = tf.shape(x)[0] # classes*query
    m = tf.shape(yw)[0] #classes*shot
    d = tf.shape(x)[1]  #features
    #n = tf.Print(n,[n],'This is n: ')
    #m = tf.Print(m,[m],'This is m: ')
    #d = tf.Print(d,[d],'This is d: ')

    #yw = tf.Print(yw,[tf.shape(yw)],'yw Info: ')
    #x = tf.Print(x,[tf.shape(x)],'x Info: ')
    x = tf.tile(tf.expand_dims(x, axis = 1), (1,m,1))
    yb = yw[:,-1]  #yw[:,-1] I think this works just like numpy now in tensor flow hopefully this is not problematic
    yw = yw[:,:-1] #yw[:,:-1]
    #yw = tf.Print(yw,[tf.shape(yw)],'This is shape of yw before: ')
    yw = tf.tile(tf.expand_dims(yw, axis = 0), (n,1,1))
    output = tf.reduce_sum(tf.multiply(x,yw), axis = 2)
    #output = tf.Print(output,[tf.shape(output)],'output Info: ') 
    
    return output

In [7]:
n_epochs = 50
n_episodes = 100
n_way = 60
n_shot = 1
n_query = 5
n_examples = 20
im_width, im_height, channels = 28, 28, 1
h_dim = 64
z_dim = 64

In [8]:
# Load Train Dataset
root_dir = './data/omniglot'
train_split_path = os.path.join(root_dir, 'splits', 'train.txt')
with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()]
n_classes = len(train_classes)
train_dataset = np.zeros([n_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(train_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        train_dataset[i, j] = im
print(train_dataset.shape)

(4112, 20, 28, 28)


In [9]:
x = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
q = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
x_shape = tf.shape(x)
q_shape = tf.shape(q)
num_classes, num_support = x_shape[0], x_shape[1]
num_queries = q_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)
z_s,emb_xs = encoder(tf.reshape(x, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
z_q,emb_xq = encoder(tf.reshape(q, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True)
alpha = compute_alpha(emb_xq,z_s)
beta = compute_alpha(emb_xs,z_q)
beta = tf.transpose(beta)
alpha = tf.reshape(alpha, [num_classes,num_queries,num_classes,num_support])
beta = tf.reshape(beta, [num_classes,num_queries,num_classes,num_support])

alpha_sum = tf.reduce_sum(alpha,axis = 3) + tf.reduce_sum(beta,axis = 3)
log_p_y = tf.reshape(tf.nn.log_softmax(alpha_sum), [num_classes, num_queries, -1])
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))

In [10]:
train_op = tf.train.AdamOptimizer().minimize(ce_loss)

In [11]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [12]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
        support = np.expand_dims(support, axis=-1)
        query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

[epoch 1/50, episode 50/100] => loss: 3.47415, acc: 0.13667
[epoch 1/50, episode 100/100] => loss: 2.98717, acc: 0.17667
[epoch 2/50, episode 50/100] => loss: 2.92872, acc: 0.21667
[epoch 2/50, episode 100/100] => loss: 2.17049, acc: 0.36667
[epoch 3/50, episode 50/100] => loss: 1.89829, acc: 0.43000
[epoch 3/50, episode 100/100] => loss: 1.86694, acc: 0.46333
[epoch 4/50, episode 50/100] => loss: 1.67447, acc: 0.52000
[epoch 4/50, episode 100/100] => loss: 1.51066, acc: 0.53333
[epoch 5/50, episode 50/100] => loss: 1.23161, acc: 0.67000
[epoch 5/50, episode 100/100] => loss: 1.29993, acc: 0.61000
[epoch 6/50, episode 50/100] => loss: 0.96550, acc: 0.74333
[epoch 6/50, episode 100/100] => loss: 0.99656, acc: 0.67000
[epoch 7/50, episode 50/100] => loss: 0.99735, acc: 0.72000
[epoch 7/50, episode 100/100] => loss: 0.95893, acc: 0.70667
[epoch 8/50, episode 50/100] => loss: 1.03146, acc: 0.69000
[epoch 8/50, episode 100/100] => loss: 1.10622, acc: 0.69000
[epoch 9/50, episode 50/100] => 

In [13]:
# Load Test Dataset
root_dir = './data/omniglot'
test_split_path = os.path.join(root_dir, 'splits', 'test.txt')
with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
n_test_classes = len(test_classes)
test_dataset = np.zeros([n_test_classes, n_examples, im_height, im_width], dtype=np.float32)
for i, tc in enumerate(test_classes):
    alphabet, character, rotation = tc.split('/')
    rotation = float(rotation[3:])
    im_dir = os.path.join(root_dir, 'data', alphabet, character)
    im_files = sorted(glob.glob(os.path.join(im_dir, '*.png')))
    for j, im_file in enumerate(im_files):
        im = 1. - np.array(Image.open(im_file).rotate(rotation).resize((im_width, im_height)), np.float32, copy=False)
        test_dataset[i, j] = im
print(test_dataset.shape)

(1692, 20, 28, 28)


In [14]:
n_test_episodes = 1000
n_test_way = 20
n_test_shot = 5
n_test_query = 15

In [15]:
print('Testing...')
avg_acc = 0.
for epi in range(n_test_episodes):
    epi_classes = np.random.permutation(n_test_classes)[:n_test_way]
    support = np.zeros([n_test_way, n_test_shot, im_height, im_width], dtype=np.float32)
    query = np.zeros([n_test_way, n_test_query, im_height, im_width], dtype=np.float32)
    for i, epi_cls in enumerate(epi_classes):
        selected = np.random.permutation(n_examples)[:n_test_shot + n_test_query]
        support[i] = test_dataset[epi_cls, selected[:n_test_shot]]
        query[i] = test_dataset[epi_cls, selected[n_test_shot:]]
    support = np.expand_dims(support, axis=-1)
    query = np.expand_dims(query, axis=-1)
    labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
    ls, ac = sess.run([ce_loss, acc], feed_dict={x: support, q: query, y:labels})
    avg_acc += ac
    if (epi+1) % 50 == 0:
        print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
avg_acc /= n_test_episodes
print('Average Test Accuracy: {:.5f}'.format(avg_acc))

Testing...
[test episode 50/1000] => loss: 0.02784, acc: 0.99333
[test episode 100/1000] => loss: 0.26551, acc: 0.96667
[test episode 150/1000] => loss: 0.26347, acc: 0.96333
[test episode 200/1000] => loss: 0.20108, acc: 0.97333
[test episode 250/1000] => loss: 0.45364, acc: 0.96333
[test episode 300/1000] => loss: 0.25119, acc: 0.95667
[test episode 350/1000] => loss: 0.46046, acc: 0.96000
[test episode 400/1000] => loss: 0.12510, acc: 0.98333
[test episode 450/1000] => loss: 0.21344, acc: 0.98667
[test episode 500/1000] => loss: 0.20805, acc: 0.96667
[test episode 550/1000] => loss: 0.11927, acc: 0.98667
[test episode 600/1000] => loss: 0.03685, acc: 0.98667
[test episode 650/1000] => loss: 0.06834, acc: 0.99000
[test episode 700/1000] => loss: 0.30693, acc: 0.97333
[test episode 750/1000] => loss: 0.19741, acc: 0.98333
[test episode 800/1000] => loss: 0.27011, acc: 0.95667
[test episode 850/1000] => loss: 0.05989, acc: 0.98667
[test episode 900/1000] => loss: 0.00070, acc: 1.00000
